In [ ]:
!pip install ucimlrepo

In [ ]:
from ucimlrepo import fetch_ucirepo
import numpy as np
import pandas as pd
from pandas.plotting import scatter_matrix
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

<h1> Perceptron <h1>

In [ ]:
class Perceptron:
    """
    Implementa um classificador Perceptron para problemas de classificação multiclasse.

    Essa implementação utiliza a função softmax para lidar com problemas de classificação multiclasse.

    Parâmetros:
        X (numpy.ndarray): Matriz de características de entrada (features).
        y (numpy.ndarray): Vetor de rótulos de classe.
        taxa_aprendizado (float): Taxa de aprendizado do algoritmo Perceptron (alpha).
        max_it (int): Número máximo de iterações do algoritmo.
        pesos_aleatorios (bool): Se True, inicializa os pesos aleatoriamente, caso contrário, inicializa com zeros.
    """
    def __init__(self, X, y, taxa_aprendizado=0.01, max_it=1000, pesos_aleatorios = False):
        """Inicia variáveis do Perceptron."""
        self.taxa_aprendizado = taxa_aprendizado
        self.max_it = max_it
        self.pesos_aleatorios = pesos_aleatorios
        self.X = X
        self.y = y
        self.W = None
        self.D = None
        self.encoder = OneHotEncoder()

    
    def treina(self, X, D):
        """
        Treina o perceptron com os dados X e D passados.
        
        Parâmetros:
            X (numpy.ndarray): Matriz de características de entrada (features).
            D (numpy.ndarray): Matriz de rótulos de saída.
        
        Retornos:
            W (numpy.ndarray): Vetor de pesos treinados.
            loss (list): Lista de valores de perda a cada iteração.
            acc (float): Precisão do modelo treinado.
            cm (numpy.ndarray): Matriz de confusão do modelo treinado.
        """
        # Salva D para uso posterior e  inicializa os pesos
        self.D = D 
        self.W = self._inicializaPesos(X, D)
        
        # Contador de iteração
        i = 1
        # Erro
        E = 1
        # Lista para armazenar o erro a cada iteração
        loss = []

        # Loop enquanto máximo de iterações não foi atingido e erro ainda existe
        while i < self.max_it and E > 0:
            # Calcula wx + b
            wxi = self.W.dot(X.T)
            # Aplica softmax
            yi = self.softmax(wxi.T)
            # Calcula erro
            ei = D - yi
            # Atualiza pesos
            self.W = self.W + self.taxa_aprendizado * ei.T.dot(X)
            # Calcula nova função de erro
            E = self.crossEntropy(D, yi)
            # Adiciona erro na lista
            loss.append(E)
            # Incrementa contador de épocas
            i += 1

        # Calcula precisão e matriz de confusão e retorna
        acc = accuracy_score(np.argmax(D,axis = 1), np.argmax(yi, axis = 1))
        cm = confusion_matrix(np.argmax(D,axis = 1), np.argmax(yi, axis = 1))
        return self.W, loss, acc, cm


    def prever(self, X):
        """
        Prevê resultado para X passado
        
        Parâmetros:
            X(numpy.ndarray): Matriz de características de entrada (features).
        
        Retornos:
            y_predicted(numpy.ndarray): Vetor de classes previstos.
        """
        # Calcula o produto matricial entre os pesos W e X 
        wxi = self.W.dot(X.T)
        # Aplica a função softmax no resultado anterior
        y = self.softmax(wxi.T)
        # Encontra o índice do maior valor em cada linha 
        y_predicted = np.argmax(y, axis = 1)
        # Retorna o vetor de predições
        return y_predicted
    
        
    def softmax(self, x):
        """
        Calcula a função softmax de um vetor de entrada x.
    
        Parâmetros:
            x (numpy.ndarray): Vetor de entrada contendo os valores a serem normalizados.
        
        Retorna:
            resp(numpy.ndarray): Vetor contendo os valores normalizados pela função softmax.
        """
        resp = []
        for i in range(len(x)):
            resp.append(np.exp(x[i])/np.sum(np.exp(x[i])))
        return np.array(resp)
    
        
    def crossEntropy(self, D, Y):
        """
        Calcula a função de perda com entropia cruzada.
    
        Parâmetros:
            D (numpy.ndarray): Valores reais (classes).
            Y (numpy.ndarray): Valores previstos.
        
        Retorna:
            float: Valor médio da perda de entropia cruzada.
        """
        # Evita instabilidades numéricas e divisões por 0
        epsilon = 1e-12
        Y = np.clip(Y, epsilon, 1 - epsilon)
        # Calcula a entropia cruzada e retorna a média
        loss = -np.sum(D * np.log(Y)) 
        return np.mean(loss)

    def getXY(self):
        """Retorna X e Y com bias, one hot encode e separados para treino, validação e teste"""
        x_bias = self._addBias()
        y_encoded = self._encode()
        return self._split(x_bias, y_encoded)

    def _split(self, X, y):
        """Separa X e y em treino, validação e teste"""
        x_treino, x_prova, y_treino, y_prova = train_test_split(X, y, test_size=0.3, random_state=1)
        x_validacao, x_teste, y_validacao, y_teste = train_test_split(x_prova, y_prova, test_size=0.5)
        return x_treino, x_validacao, x_teste, y_treino, y_validacao, y_teste
    
    def _addBias(self):
        """Adiciona bias diretamente no vetor X"""
        return np.vstack([np.ones(self.X.shape[0]), self.X.values.T]).T
    
    def _encode(self):
        """Define a classe do Perceptron como um e as demais como 0"""
        y = self.y.values.reshape(-1, 1)
        return self.encoder.fit_transform(y).toarray()
    
    def _inicializaPesos(self, X, D):
        """Inicializa vetor de pesos com valores aleatórios se a variavel pesos_aleatorios for verdadeira, 
            se não, inicializa com zeros""" 
        if(self.pesos_aleatorios):
            return np.random.uniform(-1,1,(D.shape[1], X.shape[1]))
        return np.zeros(shape = (D.shape[1], X.shape[1]), dtype = float)

    def getClasses(self):
        """Retorna classes do Encoder"""
        return self.encoder.categories_[0]

<h2> Iris Dataset <h2>

In [ ]:
# fetch dataset 
iris = fetch_ucirepo(id=53) 
  
# data (as pandas dataframes) 
X = iris.data.features 
y = iris.data.targets 

In [ ]:
# Cria um perceptron com os dados X e y
perceptron = Perceptron(X, y, pesos_aleatorios=False, taxa_aprendizado= 0.1)

# Divide os dados em conjuntos de treino, validação e teste
x_treino, x_validacao, x_teste, y_treino, y_validacao, y_teste = perceptron.getXY()

# Treina o perceptron no conjunto de treino
t_W, t_loss, t_acc, t_cm = perceptron.treina(x_treino, y_treino) 

# Faz previsões no conjunto de validação
v_pred = perceptron.prever(x_validacao)

# Calcula a precisão no conjunto de validação
v_acc = accuracy_score(np.argmax(y_validacao,axis = 1), v_pred)

# Calcula a matriz de confusão no conjunto de validação
v_cm = confusion_matrix(np.argmax(y_validacao,axis = 1), v_pred)

# Faz previsões no conjunto de teste
y_pred  = perceptron.prever(x_teste)

# Calcula a precisão no conjunto de teste
y_acc = accuracy_score(np.argmax(y_teste,axis = 1), y_pred)

# Calcula a matriz de confusão no conjunto de teste
y_cm = confusion_matrix(np.argmax(y_teste,axis = 1), y_pred)

# Printa as precisões
print('Precisão treino:', t_acc)  
print('Precisão validação:', v_acc)
print('Precisão teste:', y_acc)

In [ ]:
#Gráfico de perda
plt.plot(range(len(t_loss)), t_loss)

# Add labels and title
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training Loss')

# Show plot
plt.show()

In [ ]:
# Plot Matriz de Confusão Treino
tcm_disp = ConfusionMatrixDisplay(t_cm, display_labels=perceptron.getClasses())
tcm_disp.plot()
ax = plt.gca()
ax.set_title("Matriz de Confusão Treino")
plt.show()

In [ ]:
# Plot Matriz de Confusão Validação
vcm_disp = ConfusionMatrixDisplay(v_cm, display_labels=perceptron.getClasses())
vcm_disp.plot()
ax = plt.gca()
ax.set_title("Matriz de Confusão Validação")
plt.show()

In [ ]:
# Plot Matriz de Confusão Teste
ycm_disp = ConfusionMatrixDisplay(y_cm, display_labels=perceptron.getClasses())
ycm_disp.plot()
ax = plt.gca()
ax.set_title("Matriz de Confusão Teste")
plt.show()

<h2> Wine Dataset <h2>

In [ ]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
wine = fetch_ucirepo(id=109) 
  
# data (as pandas dataframes) 
X = wine.data.features 
y = wine.data.targets 

In [ ]:
# Normalização de dados entre 0 e 1 com Min Max Scaler
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(X)
X = pd.DataFrame(x_scaled, columns=X.columns, index=X.index)

In [ ]:
#Inclusão das classes para plotagem de graficos
df = X.copy()
df['class'] = y.values

In [ ]:
#Plotando Scatter Matrix com todas as features para determinar as que melhor separam as classes
features = df.columns.tolist()
features.remove('class')
scatter_matrix(df[features], c=df['class'], figsize=(20,20), cmap='viridis')

for ax in plt.gcf().axes:
    ax.set_xlabel(ax.get_xlabel(), rotation=90, ha='center') 
    ax.set_ylabel(ax.get_ylabel(), rotation=0, ha='right')

plt.show()

In [ ]:
#Plotando Scatter Matrix com as features que melhor separam as classes
features = df.columns.tolist()
features.remove('class')
scatter_matrix(df[['Flavanoids', 'Color_intensity', 'Total_phenols']], c=df['class'], figsize=(20,20), cmap='viridis')

for ax in plt.gcf().axes:
    ax.set_xlabel(ax.get_xlabel(), rotation=90, ha='center') 
    ax.set_ylabel(ax.get_ylabel(), rotation=0, ha='right')

plt.show()

In [ ]:
# Treino, validação e teste do Perceptron com as features escolhidas
perceptron = Perceptron(df[['Flavanoids', 'Color_intensity', 'Total_phenols']], y, 
                        pesos_aleatorios=True, taxa_aprendizado= 0.01)
x_treino, x_validacao, x_teste, y_treino, y_validacao, y_teste = perceptron.getXY()

t_W, t_loss, t_acc, t_cm = perceptron.treina(x_treino, y_treino)

v_pred = perceptron.prever(x_validacao)
v_acc = accuracy_score(np.argmax(y_validacao,axis = 1), v_pred)
v_cm = confusion_matrix(np.argmax(y_validacao,axis = 1), v_pred)

y_pred  = perceptron.prever(x_teste)
y_acc = accuracy_score(np.argmax(y_teste,axis = 1), y_pred)
y_cm = confusion_matrix(np.argmax(y_teste,axis = 1), y_pred)

# Printa precisões
print('Precisão treino:', t_acc)
print('Precisão validação:', v_acc)
print('Precisão teste:', y_acc)